In [28]:
import pandas as pd
import nltk

from nltk.stem import PorterStemmer

file = "verification results.csv"

df = pd.read_csv(file, encoding='utf-8')

# only non-relevant


df.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd9 in position 55162: invalid continuation byte

In [21]:
### Prep our corpus
corpus = ' '.join(df['text'])
corpus = corpus.lower()

# tokenize
words = nltk.tokenize.word_tokenize(corpus)

# stem
# non characters out
ps = PorterStemmer()

words = [w for w in words if w.isalpha()]

## count
word_dict = nltk.FreqDist(words)

sw = nltk.corpus.stopwords.words('english')

words_except_sw = nltk.FreqDist(w for w in words if w not in sw)

words_except_sw
#corpus[:1000]

FreqDist({'https': 166, 'victoria': 96, 'amp': 38, 'housing': 38, 'park': 26, 'camp': 22, 'people': 19, 'like': 18, 'city': 18, 'violence': 18, ...})

In [22]:
rslt = pd.DataFrame(words_except_sw.most_common(10), columns=['Word', 'Frequency'])

rslt

,Word,Frequency
0,https,166
1,victoria,96
2,amp,38
3,housing,38
4,park,26
5,camp,22
6,people,19
7,like,18
8,city,18
9,violence,18
